In [1]:
import glob
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 训练和测试数据的存储地址
train_data_path = 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train'
test_data_path = 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Test'
# 训练手写体图片的存储地址
train_image_path = glob.glob(train_data_path + '/*/*.jpg')
# 提取手写体图片的标签
train_labels = [p.split("Train\\")[1].split("\\")[0] for p in train_image_path]
# 根据计算机cpu的个数自动做并行运算
AUTOTUNE =tf.data.experimental.AUTOTUNE

In [16]:
# 将汉字转为数字标签
def convert_to_number(data_path, labels_character, split_str):
    # 读取所有汉字目录
    label_path = glob.glob(data_path + '/*')
    # 对汉字目录进行处理，获取所有的汉字列表
    label_strs = np.array([lp.split(split_str)[1] for lp in label_path])
    # 创建一个行数为图片总数，列数为1的零矩阵
    len_labels_character = len(labels_character)
    vector = np.zeros((len_labels_character, 1))
    # 对所有图片对应的汉字标签进行处理，将其一一转为对应的数字
    for i in range(len_labels_character):
        vector[i, 0] = label_strs.tolist().index(labels_character[i])
    return len(label_strs), vector

In [31]:
# 通过路径读取图片
def load_preprosess_image(path, label):
    image = tf.io.read_file(path)
    # 解码
    image = tf.image.decode_jpeg(image, channels=1)
    # 将图像处理成相同大小
    image = tf.image.resize(image, [96, 96])
    # 改变数据类型
    image = tf.cast(image, tf.float32)
    # 归一化
    image = image / 255

    #label = tf.reshape(label, (545, 10))
    # label = tf.reshape(label, (1, train_count * num_label))
    return image, label

In [32]:
# 训练手写体图片的存储地址
train_image_path = glob.glob(train_data_path + '/*/*.jpg')
# p = "D:\\Desktop\\hand writer\\CASIA_Dataset_Image\\Gnt 1.0Train\\艇\\162903.jpg"
# 提取手写体图片的标签，标签为所有汉字列表
train_image_labels = [p.split("Train\\")[1].split("\\")[0] for p in train_image_path]
# 将所有的汉字转为数字编码
num_labels, train_num_labels = convert_to_number(train_data_path, train_image_labels, "Train\\")
train_num_labels = tf.cast(train_num_labels, tf.int32)

# 创建dataset训练对象
train_image_ds = tf.data.Dataset.from_tensor_slices((train_image_path, np.array(train_num_labels)))

In [33]:
train_image_ds

<TensorSliceDataset shapes: ((), (1,)), types: (tf.string, tf.int32)>

In [34]:
train_image_ds = train_image_ds.map(load_preprosess_image, num_parallel_calls=AUTOTUNE)

In [35]:
train_image_ds

<ParallelMapDataset shapes: ((96, 96, 1), (1,)), types: (tf.float32, tf.int32)>

In [36]:
imgs, labels = next(iter(train_image_ds))

In [37]:
imgs[0]

<tf.Tensor: shape=(96, 1), dtype=float32, numpy=
array([[1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
      

In [11]:
train_image_path

['D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\伴\\103870.jpg',
 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\伴\\106603.jpg',
 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\伴\\111909.jpg',
 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\伴\\113993.jpg',
 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\伴\\119184.jpg',
 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\伴\\12330.jpg',
 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\伴\\123846.jpg',
 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\伴\\126567.jpg',
 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\伴\\130596.jpg',
 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\伴\\135899.jpg',
 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\伴\\139273.jpg',
 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\伴\\142011.jpg',
 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\伴\\144773.jpg',
 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Tr

In [12]:
test_image_path = glob.glob(test_data_path + '/*/*.jpg')
len(test_image_path)

598

In [13]:
test_image_label = [p.split("Test\\")[1].split("\\")[0] for p in test_image_path]
len(test_image_label)

598

In [14]:
test_image_path = glob.glob(test_data_path + '/*/*.jpg')
test_image_label = [p.split("Test\\")[1].split("\\")[0] for p in test_image_path]
# 将所有的汉字标签列表转为one-hot编码
num, test_label_onehot = convert_to_one_hot(test_data_path, test_image_label, "Test\\")
# 创建dataset训练对象
#test_image_ds = tf.data.Dataset.from_tensor_slices((test_image_path, test_label_onehot))

NameError: name 'convert_to_one_hot' is not defined

In [ ]:
test_image_ds = tf.data.Dataset.from_tensor_slices((test_image_path, test_label_onehot))

In [ ]:
len(test_image_ds)

In [ ]:
def convert_to_one_hot(data_path, labels, split_str):
    # 读取所有汉字目录
    label_path = glob.glob(data_path + '/*')
    # 对汉字目录进行处理，生成汉字标签列表
    label_strs = np.array([lp.split(split_str)[1] for lp in label_path])
    # 创建一个存放one-hot标签的二维零矩阵
    vector = np.zeros((len(labels), len(label_path)))
    # 对所有的汉字标签进行处理，将其一一转为one-hot编码
    for i in range(len(labels)):
        vector[i, label_strs.tolist().index(labels[i])] = 1
    return len(label_strs),vector

In [ ]:
vector = convert_to_one_hot(train_labels)

In [ ]:
len(vector)

In [ ]:
train_image_ds = tf.data.Dataset.from_tensor_slices((train_image_path,train_labels))

In [ ]:
len(train_image_ds)

In [ ]:
train_image_ds = train_image_ds.map(load_preprosess_image,num_parallel_calls=AUTOTUNE)

In [ ]:
train_image_ds

In [ ]:
model= tf.keras.Sequential() # 顺序模型
# 第一层使用卷积层，卷积层提取图片特征的能力强   32-fliter个数  (3,3)-卷积核大小，一般使用3*3或5*5  input_shape-图片大小
model.add(tf.keras.layers.Conv2D(64,(3,3),input_shape=train_images.shape[1:],activation='relu',padding='same'))
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'))
# 最大池化层
model.add(tf.keras.layers.MaxPool2D())
# 抑制过拟合
model.add(tf.keras.layers.Dropout(0.5))
# 卷积核以2的指数增长
model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='same'))
# 最大池化层
model.add(tf.keras.layers.MaxPool2D())
# 抑制过拟合
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Conv2D(256,(3,3),activation='relu',padding='same'))
model.add(tf.keras.layers.Conv2D(256,(3,3),activation='relu',padding='same'))
# 抑制过拟合
model.add(tf.keras.layers.Dropout(0.5))
# 全局平均池化：得到一个二维的数据
model.add(tf.keras.layers.GlobalAveragePooling2D())
# Dense 输出层：输出一个维度为10的数据  softmax:输出概率值
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='softmax'))

In [ ]:
p = ['D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\呸\\101968.jpg', 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\呸\\10254.jpg', 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\呸\\104702.jpg', 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\呸\\110010.jpg', 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\呸\\115940.jpg', 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\呸\\117395.jpg', 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\呸\\121952.jpg', 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\呸\\124668.jpg', 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\呸\\128689.jpg', 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\呸\\134003.jpg', 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\呸\\137368.jpg', 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\呸\\14283.jpg', 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\呸\\143968.jpg', 'D:/Desktop/hand writer/dataset/test_CASIA_Image/Train\\呸\\146730.jpg']

In [ ]:
l = [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((p,l))

In [ ]:
ds